In [20]:
import pandas as pd 
import os 
import numpy
import json 
from tagging_util import Tagger 
import six

In [21]:
# training_commit_samples.csv, bug_doc_clean.json 

# Tagging

In [22]:
df = pd.read_csv("training_cmt_preprocessed_final.csv")
print(df.shape)

(61745, 13)


In [23]:
with open("bug_doc_clean.json", "r") as f: 
    bug_doc_clean = json.load(f)

for k in bug_doc_clean.keys(): 
    print(k)

Bug fix
Documentation
Clean up


In [24]:
prelabel_bug_doc_clean = list(bug_doc_clean.keys())
print(prelabel_bug_doc_clean)

['Bug fix', 'Documentation', 'Clean up']


In [25]:
tagger = Tagger()
# df = tagger.prelabeling(df, "bug_doc_clean.json", prelabel_bug_doc_clean)

In [26]:
bugfix = df.index[df["Bug fix"] == 1].tolist()
print(len(bugfix))

2517


In [27]:
link = tagger.get_commit(df, "Bug fix", 1)
print(link)

https://github.com/apache/avro/commit/37ce7f72478fcd3dd15637efe25b422f9631bacc


In [9]:
tagger.getlabels(df, "289ea57aa60c151f4fd2eb59c73b8d81c141d2b8")

testing
Bug fix
Documentation


In [10]:
df = tagger.label(df, "Documentation", "289ea57aa60c151f4fd2eb59c73b8d81c141d2b8", 0)
tagger.getlabels(df, "289ea57aa60c151f4fd2eb59c73b8d81c141d2b8")
df = tagger.check(df, "289ea57aa60c151f4fd2eb59c73b8d81c141d2b8")

testing
Bug fix


In [19]:
tagger.writing_to_csv(df, "training_cmt_preprocessed_final.csv")